# 1. Merge the datasets such that the resultant dataset contains only the intersection of rows present in both files.

In [33]:
import pandas as pd
import statsmodels.api as sm  

In [4]:
urban_pop = pd.read_csv("share-of-population-urban.csv")
tax_pct_gdp = pd.read_csv("taxes-on-incomes-of-individuals-and-corporations-gdp.csv")

In [5]:
urban_pop.head()

,Entity,Code,Year,Urban_Population
0,Afghanistan,AFG,1960,8.401
1,Afghanistan,AFG,1961,8.684
2,Afghanistan,AFG,1962,8.976
3,Afghanistan,AFG,1963,9.276
4,Afghanistan,AFG,1964,9.586


In [6]:
tax_pct_gdp.head()

,Entity,Code,Year,Tax_Percent_GDP
0,Afghanistan,AFG,2003,0.165953
1,Afghanistan,AFG,2004,0.411647
2,Afghanistan,AFG,2005,0.320864
3,Afghanistan,AFG,2006,1.261181
4,Afghanistan,AFG,2007,1.323461


In [14]:
# Merge the 2 dataframes, only including rows where there is both a tax value and a population value for a given combination of code and year.

social_data = pd.merge(urban_pop, tax_pct_gdp, on = ['Code', 'Year'], how = 'inner')

In [16]:
social_data.head(20)

,Entity_x,Code,Year,Urban_Population,Entity_y,Tax_Percent_GDP
0,Afghanistan,AFG,2003,22.353,Afghanistan,0.165953
1,Afghanistan,AFG,2004,22.5,Afghanistan,0.411647
2,x,AFG,2005,22.703,Afghanistan,0.320864
3,Afghanistan,AFG,2006,22.907,Afghanistan,1.261181
4,Afghanistan,AFG,2007,23.113,Afghanistan,1.323461
5,Afghanistan,AFG,2008,23.32,Afghanistan,1.664719
6,Afghanistan,AFG,2009,23.528,Afghanistan,2.608631
7,Afghanistan,AFG,2010,23.737,Afghanistan,2.563413
8,Afghanistan,AFG,2011,23.948,Afghanistan,2.573097
9,Afghanistan,AFG,2012,24.16,Afghanistan,2.477800


In [42]:
# Drop 'Entity_y' column b/c that info already exists in 'Entity_x'
merged_social_data = social_data.drop('Entity_y', 1) 

# Rename 'Entity_x' to 'Entity'
merged_social_data1 = merged_social_data.rename(columns={'Entity_x': 'Entity'}) 

# Some string values were found in the Urban_Population data (e.g. '43%'). This poses an error when running OLS regression.
# Thus, '%' was stripped from those values, and the remaining number was converted to a float to match the rest of the column.

merged_social_data1['Urban_Population'] = merged_social_data1['Urban_Population'].str.rstrip('%').astype('float')

# As a consistency measure, both relevant columns (urban pop and tax percent) were cast as float.
merged_social_data1['Tax_Percent_GDP'] = merged_social_data1['Tax_Percent_GDP'].astype('float')

merged_social_data1.head(20)

,Entity,Code,Year,Urban_Population,Tax_Percent_GDP
0,Afghanistan,AFG,2003,22.353,0.165953
1,Afghanistan,AFG,2004,22.500,0.411647
2,x,AFG,2005,22.703,0.320864
3,Afghanistan,AFG,2006,22.907,1.261181
4,Afghanistan,AFG,2007,23.113,1.323461
5,Afghanistan,AFG,2008,23.320,1.664719
6,Afghanistan,AFG,2009,23.528,2.608631
7,Afghanistan,AFG,2010,23.737,2.563413
8,Afghanistan,AFG,2011,23.948,2.573097
9,Afghanistan,AFG,2012,24.160,2.477800


# 2. Export the results to a new file called merged_social_data.csv

In [43]:
merged_social_data1.to_csv("merged_social_data.csv")

# 3. Using the merged results, generate an ordinary least squares regression showing the effect of the independent variable 'Urban_Population' on the dependent variable 'Tax_Percent_GDP'. Ensure results contain: A. R Squared. B. t-statistic and p-value of the coefficient and intercept. C. Degrees of freedom. D. The spread of the residuals.

In [56]:
# Assign independent variable as X, and dependent variable as Y

X = merged_social_data1['Urban_Population']
Y = merged_social_data1['Tax_Percent_GDP']

# Including constant is required for running regression
X = sm.add_constant(X)

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [57]:
mod = sm.OLS(Y, X).fit()

In [58]:
mod_info = mod.summary()
print(mod_info)

                            OLS Regression Results                            
Dep. Variable:        Tax_Percent_GDP   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     1127.
Date:                Fri, 22 May 2020   Prob (F-statistic):          3.38e-222
Time:                        16:58:19   Log-Likelihood:                -14029.
No. Observations:                4827   AIC:                         2.806e+04
Df Residuals:                    4825   BIC:                         2.808e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                1.6963      0.158  